In [1]:
from fileinput import filename
from typing import List, Optional
from pathlib import Path
import pandas as pd
import numpy as np
import pdfplumber
import re
import os
from datetime import datetime
import logging
import sys

In [9]:
DIR_DATA = Path.cwd().parents[0] / "data"
# DIR_DATA = script_dir.parent / "data"
DIR_REPORTS_PDF = DIR_DATA / "raw_pdfs/mse-daily-reports"
DIR_REPORTS_CSV = DIR_DATA / "csv_files/mse-daily-data"
# DIR_REPORTS_PDF = DIR_DATA / "mse-daily-reports"
# DIR_REPORTS_CSV = DIR_DATA / "mse-daily-data"
# FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_17_July_2018.pdf"
# FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_01_December_2022.pdf"
# FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_01_December_2021.pdf"
FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_01_October_2020.pdf"
# FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_01_February_2019.pdf"
# FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_13_August_2018.pdf"
# FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_08_August_2017.pdf"
# FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_16_May_2023.pdf"
# FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_02_July_2018.pdf"
# FILE_PDF = DIR_REPORTS_PDF / "Daily_Report_12_April_2024.pdf"
# FILE_PDF = DIR_REPORTS_PDF / "Daily 08 September 2025.pdf"
FILE_CSV = DIR_REPORTS_CSV / "mse-daily-09-05-2025.csv"

In [3]:
import re
from datetime import date, time, datetime
from pathlib import Path
import pdfplumber

# Month map (handles "Sep" and "Sept")
_MONTHS = {
    'jan':1,'january':1,'feb':2,'february':2,'mar':3,'march':3,'apr':4,'april':4,
    'may':5,'jun':6,'june':6,'jul':7,'july':7,'aug':8,'august':8,
    'sep':9,'sept':9,'september':9,'oct':10,'october':10,
    'nov':11,'november':11,'dec':12,'december':12
}

def _mkdate(y, m, d):  # y,m,d may be str
    return date(int(y), int(m), int(d))

def _norm_text(s: str) -> str:
    return re.sub(r'\s+', ' ', s or '').strip()

def _parse_date_str(s: str, day_first: bool = True):
    """Parse a date from free text. Returns datetime.date or None."""
    s = _norm_text(s)

    # 1) 5 September 2025 | 05 Sep 2025 | 5 Sept, 2025 | 5th September 2025
    m = re.search(r'(?i)\b(\d{1,2})(?:st|nd|rd|th)?\s+([A-Za-z]{3,9}),?\s+(20\d{2})\b', s)
    if m:
        d, mon, y = m.groups()
        mon_num = _MONTHS.get(mon.lower())
        if mon_num:
            return _mkdate(y, mon_num, d)

    # 2) September 5, 2025 | Sep 05 2025 | Sept 5th 2025
    m = re.search(r'(?i)\b([A-Za-z]{3,9})\s+(\d{1,2})(?:st|nd|rd|th)?,?\s+(20\d{2})\b', s)
    if m:
        mon, d, y = m.groups()
        mon_num = _MONTHS.get(mon.lower())
        if mon_num:
            return _mkdate(y, mon_num, d)

    # 3) ISO-like: 2025-09-05 / 2025/09/05 / 2025.09.05
    m = re.search(r'\b(20\d{2})[-/.](\d{1,2})[-/.](\d{1,2})\b', s)
    if m:
        y, mth, d = m.groups()
        try: return _mkdate(y, mth, d)
        except ValueError: pass

    # 4) Numeric: 05-09-2025 | 05/09/2025 | 5.9.2025
    m = re.search(r'\b(\d{1,2})[-/.](\d{1,2})[-/.](20\d{2})\b', s)
    if m:
        a, b, y = m.groups()
        # day-first by default (MSE style)
        d, mth = (a, b) if day_first else (b, a)
        try: return _mkdate(y, mth, d)
        except ValueError: pass

    return None

def _parse_time_str(s: str):
    """Parse a time from free text. Returns datetime.time or None."""
    s = _norm_text(s)

    # 12-hour with seconds or without (e.g., 02:39:52 pm, 2:39 pm)
    m = re.search(r'(?i)\b(\d{1,2}):(\d{2})(?::(\d{2}))?\s*(am|pm)\b', s)
    if m:
        hh, mm, ss, ap = m.groups()
        hh, mm, ss = int(hh), int(mm), int(ss or 0)
        ap = ap.lower()
        if hh == 12: hh = 0
        if ap == 'pm': hh += 12
        try: return time(hh, mm, ss)
        except ValueError: return None

    # 24-hour with optional seconds (e.g., 14:39:52 or 14:39)
    m = re.search(r'\b([01]?\d|2[0-3]):([0-5]\d)(?::([0-5]\d))\b', s)
    if m:
        hh, mm, ss = map(int, m.groups())
        try: return time(hh, mm, ss)
        except ValueError: return None

    m = re.search(r'\b([01]?\d|2[0-3]):([0-5]\d)\b', s)
    if m:
        hh, mm = map(int, m.groups())
        try: return time(hh, mm)
        except ValueError: return None

    return None

def extract_print_date_time(pdf_path: str | Path, search_pages: int = 2, day_first: bool = True):
    """
    Extract ONLY the 'Print Date' and 'Print Time' from the PDF text.

    Returns
    -------
    {
      'date': datetime.date | None,
      'time': datetime.time | None,
      'raw_date': str | None,  # snippet matched after the label (if any)
      'raw_time': str | None
    }
    """
    pdf_path = Path(pdf_path)
    raw_date_snip = raw_time_snip = None
    text = ""

    with pdfplumber.open(pdf_path) as pdf:
        n = min(max(search_pages, 1), len(pdf.pages))
        # Concatenate small chunks (keeps label context)
        page_texts = []
        for i in range(n):
            page_texts.append(pdf.pages[i].extract_text() or "")
        text = "\n".join(page_texts)

    # Prefer labeled fields
    m = re.search(r'(?is)Print\s*Date\s*:?\s*([^\n\r]+)', text)
    if m: raw_date_snip = m.group(1)
    m = re.search(r'(?is)Print\s*Time\s*:?\s*([^\n\r]+)', text)
    if m: raw_time_snip = m.group(1)

    d = _parse_date_str(raw_date_snip) if raw_date_snip else _parse_date_str(text)
    t = _parse_time_str(raw_time_snip) if raw_time_snip else _parse_time_str(text)

    return {'date': d, 'time': t, 'raw_date': (raw_date_snip or None), 'raw_time': (raw_time_snip or None)}

In [4]:
# --- Example usage ---
info = extract_print_date_time(FILE_PDF)
print("Date:", info['date'])
print("Time:", info['time'])


Date: 2017-08-08
Time: 14:30:00


In [5]:
def to_numeric_clean(val):
    """
    Clean and convert a value to numeric:
    - None/empty -> NaN
    - (123.45) -> -123.45
    - remove commas
    """
    if val is None:
        return np.nan
    val = str(val).strip()
    if val.lower() == "none" or val == "":
        return np.nan
    # Handle parentheses as negatives
    if val.startswith("(") and val.endswith(")"):
        val = "-" + val[1:-1]
    # Remove commas
    val = val.replace(",", "")
    try:
        return float(val)
    except ValueError:
        return np.nan

def clean_cell(x):
    if x is None:
        return None
    x = re.sub(r'\s+', ' ', str(x)).strip()
    x = x.replace('–', '-').replace('—', '-')
    return x if x else '-'

def is_numericish(s: Optional[str]) -> bool:
    if s is None:
        return False
    s = str(s).strip().replace(",", "")
    return bool(re.fullmatch(r"[-+]?(\d+(\.\d+)?|\.\d+)(%?)", s))

def is_header_like(row: list) -> bool:
    """Header-like = many text cells, few numeric cells."""
    cells = [c for c in row if c is not None and str(c).strip() != ""]
    if not cells:
        return False
    num_numeric = sum(1 for c in cells if is_numericish(c))
    num_alpha   = sum(1 for c in cells if re.search(r"[A-Za-z]", str(c)))
    return (num_alpha >= max(1, len(cells)//4)) and (num_numeric / len(cells) <= 0.5)


def remove_text_rows(rows, expected_length=None):
    """
    Remove rows whose first cell looks like a word/sentence (non-numeric)
    and optionally skip rows shorter than the expected length.

    Args:
        rows (list[list]): List of rows (each row is a list of cell values).
        expected_length (int, optional): Minimum expected number of cells in a valid row.

    Returns:
        list[list]: Filtered list with only valid rows.
    """
    filtered = []
    for row in rows:
        # Skip empty rows
        if not row:
            continue

        # Skip rows shorter than expected length (if specified)
        if expected_length is not None and len(row) < expected_length:
            print(f"Skipped short row (len={len(row)} < {expected_length}): {row}")
            continue

        first = str(row[0]).strip()

        # Skip rows whose first cell contains letters (text-like)
        if re.search(r"[A-Za-z]", first):
            print("Skipped text-like first cell:", first)
            continue

        # Otherwise, keep the row
        filtered.append(row)

    return filtered


# def remove_text_rows(rows):
#     """
#     Remove rows whose first cell looks like a word or sentence (non-numeric).

#     Args:
#         rows (list[list]): The list of rows (each row is a list of cell values)

#     Returns:
#         list[list]: Filtered list with only rows that start with numeric-like values.
#     """
#     filtered = []
#     for row in rows:
#         if not row:  # skip completely empty rows
#             continue
#         first = str(row[0]).strip()

#         # Check if the first value looks like text
#         # - Contains any letter (A-Z or a-z)
#         # - Or is not a number/pure symbol
#         if re.search(r'[A-Za-z]', first):
#             print("here is the first value of row skipped:", first)
#             continue  # skip rows where first cell looks like text

#         filtered.append(row)

#     return filtered

def normalize_to_width(rows: list[list], width: int) -> list[list]:
    # Step 0: Replace cells that contain only spaces with '-'
    rows = [
        [('-' if isinstance(x, str) and x.strip() == '' else x) for x in sub]
        for sub in rows
    ]

    # Step 1: Remove None values from all sublists
    data_rows = [
        [x for x in sub if x is not None]
        for sub in rows
        if any(x is not None for x in sub)
    ]

    print("first row of data_rows:", data_rows[0])
    print(f"the length of columns is {width} while the length of data_rows is {len(data_rows[0])}")
    data_rows = [
        sub[:width] if len(sub) > width else sub + [""] * (width - len(sub))
        for sub in data_rows
    ]
    # Step 2: If first row has fewer elements than width, add numbering
    if data_rows and len(data_rows[0]) < width:
        data_rows = [
            [i + 1] + sub
            for i, sub in enumerate(data_rows)
        ]

    return data_rows


# def normalize_to_width(rows: list[list], width: int) -> list[list]:
#     normalized_rows = []

#     for sub in rows:
#         # Step 0: Replace cells that contain only spaces with '-'
#         sub = [("-" if isinstance(x, str) and x.strip() == "" else x) for x in sub]

#         # Step 1: Remove None values
#         sub = [x for x in sub if x is not None]

#         # Step 2: Split company + MSE code if merged (usually index 3)
#         if len(sub) > 3 and isinstance(sub[3], str) and " " in sub[3]:
#             name_value = sub[3].rsplit(" ", 1)
#             sub[3] = name_value[0]
#             sub.insert(4, name_value[1])

#         # Step 3: Find the first cell containing multiple numbers (long numeric string)
#         for i, val in enumerate(sub):
#             if isinstance(val, str) and re.search(r"\d", val) and " " in val:
#                 # Split numbers, including negative, decimal, commas, parentheses
#                 split_nums = re.findall(r"\(?-?\d[\d,\.]*\)?", val)
#                 sub = sub[:i] + split_nums + sub[i + 1 :]
#                 break  # only split the first long numeric string

#         # Step 4: Pad or truncate to width
#         if len(sub) < width:
#             sub += [""] * (width - len(sub))
#         elif len(sub) > width:
#             sub = sub[:width]

#         normalized_rows.append(sub)

#     return normalized_rows



def extract_first_table(
    pdf_path: str | Path,
    out_csv: Optional[str | Path] = None,
    header: Optional[List[str]] = None,
    skip_header_rows: int = 0,
    auto_skip_header_like: bool = True,
) -> pd.DataFrame:
    """
    Extract the first table. If `header` is provided, we will:
      - optionally auto-skip any header-like rows at the top
      - then force DataFrame columns to `header`

    Parameters
    ----------
    pdf_path : str | Path
    out_csv : str | Path, optional
    header : List[str], optional
        Hardcoded column names to use.
    skip_header_rows : int
        Force skipping this many rows from the top of the table before data.
    auto_skip_header_like : bool
        If True, skip leading header-like rows automatically.

    Returns
    -------
    pandas.DataFrame
    """
    pdf_path = Path(pdf_path)
    out_csv = Path(out_csv) if out_csv else None

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Try a few strategies to find tables
            strategies = [
                dict(
                    vertical_strategy="lines",
                    horizontal_strategy="lines",
                    snap_tolerance=10,
                    join_tolerance=3,
                    edge_min_length=1,
                ),
                # dict(
                #     vertical_strategy="lines_strict", horizontal_strategy="lines_strict"
                # ),
                # dict(vertical_strategy="text", horizontal_strategy="text"),
                # dict(vertical_strategy="lines", horizontal_strategy="lines"),
                # dict(vertical_strategy="lines", horizontal_strategy="text"),  # mixed
                # dict(vertical_strategy="text", horizontal_strategy="lines"),
            ]
            tables = []
            for ts in strategies:
                try:
                    t = page.extract_tables(table_settings=ts) or []
                    for raw in t:
                        if raw and len(raw) >= 2 and max(len(r) for r in raw) >= 2:
                            tables.append(raw)
                    if tables:
                        break
                except Exception:
                    continue

            if not tables:
                continue

            # Use the first table found
            raw = tables[0]
            print("Original99999999999999999999999999999999999999999999999")
            print(raw)
            print("Original99999999999999999999999999999999999999999999999")
            rows = [[clean_cell(c) for c in row] for row in raw]

            rows = [r for r in rows if any(c for c in r)]

            if not rows:
                continue

            # Decide how many rows to skip from top if header is provided
            start_idx = 0
            if header:
                if auto_skip_header_like:
                    # Skip all consecutive header-like rows from the top
                    auto_skip = 0
                    for r in rows:
                        if is_header_like(r):
                            auto_skip += 1
                        else:
                            break
                    start_idx = auto_skip
                # Ensure at least skip_header_rows are skipped
                start_idx = max(start_idx, skip_header_rows)
                cols = list(header)
            else:
                # Fallback: auto-detect header = first non-empty row
                detected = rows[0]
                start_idx = 1
                cols = []
                seen = {}
                for i, name in enumerate(detected):
                    name = name or f"col_{i+1}"
                    name = re.sub(r"\s+", " ", name).strip()
                    if name in seen:
                        seen[name] += 1
                        name = f"{name}_{seen[name]}"
                    else:
                        seen[name] = 1
                    cols.append(name)
            print(f"skipped rows: {start_idx}, cols: {cols}")
            print(
                "************************************************************************"
            )
            print(rows)
            print(
                "************************************************************************"
            )
            # Remove all text-like rows
            rows = remove_text_rows(rows)
            # Build DataFrame
            data_rows = normalize_to_width(rows, len(cols))
            print("DATA ROWS",data_rows)

            df = pd.DataFrame(data_rows, columns=cols).dropna(how="all")
            print(df.counter.unique())

            # # Drop last row as it contains weighted averages
            # df = df.iloc[:-1] if len(df) > 1 else df
            # print(df.counter.unique())
            # Rearrange columns
            cols = [
                "counter",
                "daily_range_high",
                "daily_range_low",
                "buy",
                "sell",
                "previous_closing_price",
                "today_closing_price",
                "volume_traded",
                "dividend_mk",
                "dividend_yield_pct",
                "earnings_yield_pct",
                "pe_ratio",
                "pbv_ratio",
                "market_capitalization_mkmn",
                "profit_after_tax_mkmn",
                "num_shares_issue",
            ]
            df = df[cols]

            # Convert to numeric where possible
            for c in df.columns:
                if c != "counter":  # leave counter as string
                    df[c] = df[c].apply(to_numeric_clean)
            print(df.head(2))
            if out_csv:
                df.to_csv(out_csv, index=False)
                print(f"✅ First table extracted and saved to {out_csv}")
            return df

    print("⚠️ No table found in PDF.")
    return pd.DataFrame()

In [6]:
COLS = ['ser_no', 'daily_range_high', 'daily_range_low', 
         'counter', 'buy', 'sell', 'previous_closing_price', 
        'today_closing_price', 'volume_traded', 'dividend_mk', 'dividend_yield_pct',
        'earnings_yield_pct', 'pe_ratio', 'pbv_ratio', 'market_capitalization_mkmn',
        'profit_after_tax_mkmn', 'num_shares_issue']

# COLS = [ 'daily_range_high', 'daily_range_low', 
#          'counter', 'buy', 'sell', 'previous_closing_price', 
#         'today_closing_price', 'volume_traded', 'dividend_mk', 'dividend_yield_pct',
#         'earnings_yield_pct', 'pe_ratio', 'pbv_ratio', 'market_capitalization_mkmn',
#         'profit_after_tax_mkmn', 'num_shares_issue']



In [10]:
# Run the function
df = extract_first_table(FILE_PDF, header=COLS)

Original99999999999999999999999999999999999999999999999
[['No.', 'Daily Range', None, 'MSE Code', 'BUY', 'SELL', 'Previous \nClosing \nPrice', 'Today \nClosing \nPrice', 'Volume', 'Dividend\nYield\nNet\n(%)', None, 'Earnings \nYield \n(%)', 'P/E \nRatio', 'P/BV \nRatio', 'Market \nCapitalis\nation \n(MKmn)', 'After \nTax \nProfit\n(MKmn)', None], [None, 'High Low', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], ['1', '30.00', '30.00', 'AIRTEL', '28.81', '30.00', '30.00', '30.00', '16,444', '1.25', '4.17', '4.82', '20.74', '20.95', '330,000.00', '15,908.00', '11,000,000,000'], ['2', '', '', 'BHL', '0.00', '12.94', '12.94', '12.94', '0', '0.20', '1.55', '4.29', '23.28', '1.67', '10,866.37', '466.70', '839,750,705'], ['3', '12.00', '12.00', 'FDHB', '0.00', '12.49', '12.06', '12.00', '1,269,828', '0.00', '0.00', '9.47', '10.55', '3.65', '82,812.38', '7,846.36', '6,901,031,250'], ['4', '', '', 'FMBCH', '23.00', '24.00', '24.00', '24.00', '0', '0.0

In [11]:
df

,counter,daily_range_high,daily_range_low,buy,sell,previous_closing_price,today_closing_price,volume_traded,dividend_mk,dividend_yield_pct,earnings_yield_pct,pe_ratio,pbv_ratio,market_capitalization_mkmn,profit_after_tax_mkmn,num_shares_issue
0,AIRTEL,30.00,30.00,28.81,30.00,30.00,30.00,16444.0,1.25,4.17,4.82,20.74,20.95,330000.00,15908.00,1.100000e+10
1,BHL,NaN,NaN,0.00,12.94,12.94,12.94,0.0,0.20,1.55,4.29,23.28,1.67,10866.37,466.70,8.397507e+08
2,FDHB,12.00,12.00,0.00,12.49,12.06,12.00,1269828.0,0.00,0.00,9.47,10.55,3.65,82812.38,7846.36,6.901031e+09
3,FMBCH,NaN,NaN,23.00,24.00,24.00,24.00,0.0,0.00,0.00,-6.24,-16.03,1.02,58998.00,-3679.93,2.458250e+09
4,ICON,12.49,12.49,0.00,12.49,12.49,12.49,21000.0,0.21,1.68,8.69,11.51,1.13,83433.20,7248.62,6.680000e+09
5,ILLOVO,NaN,NaN,68.50,0.00,80.50,80.50,0.0,0.50,0.62,17.56,5.70,0.81,57432.27,10083.00,7.134444e+08
6,MPICO,NaN,NaN,0.00,23.95,24.00,24.00,0.0,0.25,1.04,10.03,9.97,1.41,55153.14,5532.02,2.298047e+09
7,NBM,NaN,NaN,0.00,618.75,618.75,618.75,0.0,17.91,2.89,5.92,16.89,2.87,288914.01,17106.00,4.669317e+08
8,NBS,21.00,21.00,0.00,21.00,19.44,21.00,32906.0,0.35,1.67,7.29,13.71,3.98,61122.04,4457.61,2.910573e+09
9,NICO,48.00,48.00,0.00,48.00,48.00,48.00,5817.0,1.90,3.96,16.00,6.25,1.57,50065.97,8011.00,1.043041e+09


In [28]:
df.dtypes

counter                        object
daily_range_high              float64
daily_range_low               float64
buy                           float64
sell                          float64
previous_closing_price        float64
today_closing_price           float64
volume_traded                 float64
dividend_mk                   float64
dividend_yield_pct            float64
earnings_yield_pct            float64
pe_ratio                      float64
pbv_ratio                     float64
market_capitalization_mkmn    float64
profit_after_tax_mkmn         float64
num_shares_issue              float64
dtype: object

In [12]:
def weighted_average(df: pd.DataFrame, value_col: str, weight_col: str) -> float:
    """
    Compute a weighted average for any numeric column in a DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the data.
    value_col : str
        Column name with the values of interest (e.g., 'dividend_mk', 'dividend_pct').
    weight_col : str
        Column name with the weights (e.g., 'volume_traded').

    Returns
    -------
    float
        Weighted average of the value_col, or NaN if weights sum to 0.
    """
    values = df[value_col].astype(float)
    weights = df[weight_col].astype(float)

    total_weight = weights.sum()
    if total_weight == 0:
        return np.nan

    return (values * weights).sum() / total_weight

In [13]:
cols_weighted = ['dividend_mk', 'dividend_yield_pct', 'earnings_yield_pct',
                 'pe_ratio', 'pbv_ratio', 'market_capitalization_mkmn',
                 'profit_after_tax_mkmn', 'num_shares_issue']

for col in cols_weighted:
    wa = weighted_average(df, col, 'volume_traded')
    print(f"Weighted average of {col}: {wa:.4f}")

Weighted average of dividend_mk: 0.0353
Weighted average of dividend_yield_pct: 0.1351
Weighted average of earnings_yield_pct: 9.3759
Weighted average of pe_ratio: 10.7481
Weighted average of pbv_ratio: 3.8211
Weighted average of market_capitalization_mkmn: 85170.1616
Weighted average of profit_after_tax_mkmn: 7853.3887
Weighted average of num_shares_issue: 6824787170.1106


In [14]:
df

,counter,daily_range_high,daily_range_low,buy,sell,previous_closing_price,today_closing_price,volume_traded,dividend_mk,dividend_yield_pct,earnings_yield_pct,pe_ratio,pbv_ratio,market_capitalization_mkmn,profit_after_tax_mkmn,num_shares_issue
0,AIRTEL,30.00,30.00,28.81,30.00,30.00,30.00,16444.0,1.25,4.17,4.82,20.74,20.95,330000.00,15908.00,1.100000e+10
1,BHL,NaN,NaN,0.00,12.94,12.94,12.94,0.0,0.20,1.55,4.29,23.28,1.67,10866.37,466.70,8.397507e+08
2,FDHB,12.00,12.00,0.00,12.49,12.06,12.00,1269828.0,0.00,0.00,9.47,10.55,3.65,82812.38,7846.36,6.901031e+09
3,FMBCH,NaN,NaN,23.00,24.00,24.00,24.00,0.0,0.00,0.00,-6.24,-16.03,1.02,58998.00,-3679.93,2.458250e+09
4,ICON,12.49,12.49,0.00,12.49,12.49,12.49,21000.0,0.21,1.68,8.69,11.51,1.13,83433.20,7248.62,6.680000e+09
5,ILLOVO,NaN,NaN,68.50,0.00,80.50,80.50,0.0,0.50,0.62,17.56,5.70,0.81,57432.27,10083.00,7.134444e+08
6,MPICO,NaN,NaN,0.00,23.95,24.00,24.00,0.0,0.25,1.04,10.03,9.97,1.41,55153.14,5532.02,2.298047e+09
7,NBM,NaN,NaN,0.00,618.75,618.75,618.75,0.0,17.91,2.89,5.92,16.89,2.87,288914.01,17106.00,4.669317e+08
8,NBS,21.00,21.00,0.00,21.00,19.44,21.00,32906.0,0.35,1.67,7.29,13.71,3.98,61122.04,4457.61,2.910573e+09
9,NICO,48.00,48.00,0.00,48.00,48.00,48.00,5817.0,1.90,3.96,16.00,6.25,1.57,50065.97,8011.00,1.043041e+09


In [15]:
df.reset_index(inplace=True)


In [16]:
df

,index,counter,daily_range_high,daily_range_low,buy,sell,previous_closing_price,today_closing_price,volume_traded,dividend_mk,dividend_yield_pct,earnings_yield_pct,pe_ratio,pbv_ratio,market_capitalization_mkmn,profit_after_tax_mkmn,num_shares_issue
0,0,AIRTEL,30.00,30.00,28.81,30.00,30.00,30.00,16444.0,1.25,4.17,4.82,20.74,20.95,330000.00,15908.00,1.100000e+10
1,1,BHL,NaN,NaN,0.00,12.94,12.94,12.94,0.0,0.20,1.55,4.29,23.28,1.67,10866.37,466.70,8.397507e+08
2,2,FDHB,12.00,12.00,0.00,12.49,12.06,12.00,1269828.0,0.00,0.00,9.47,10.55,3.65,82812.38,7846.36,6.901031e+09
3,3,FMBCH,NaN,NaN,23.00,24.00,24.00,24.00,0.0,0.00,0.00,-6.24,-16.03,1.02,58998.00,-3679.93,2.458250e+09
4,4,ICON,12.49,12.49,0.00,12.49,12.49,12.49,21000.0,0.21,1.68,8.69,11.51,1.13,83433.20,7248.62,6.680000e+09
5,5,ILLOVO,NaN,NaN,68.50,0.00,80.50,80.50,0.0,0.50,0.62,17.56,5.70,0.81,57432.27,10083.00,7.134444e+08
6,6,MPICO,NaN,NaN,0.00,23.95,24.00,24.00,0.0,0.25,1.04,10.03,9.97,1.41,55153.14,5532.02,2.298047e+09
7,7,NBM,NaN,NaN,0.00,618.75,618.75,618.75,0.0,17.91,2.89,5.92,16.89,2.87,288914.01,17106.00,4.669317e+08
8,8,NBS,21.00,21.00,0.00,21.00,19.44,21.00,32906.0,0.35,1.67,7.29,13.71,3.98,61122.04,4457.61,2.910573e+09
9,9,NICO,48.00,48.00,0.00,48.00,48.00,48.00,5817.0,1.90,3.96,16.00,6.25,1.57,50065.97,8011.00,1.043041e+09


In [17]:
int(df.volume_traded.sum())

1345995

In [18]:
cols = ['ser_no', 'daily_range_high', 'daily_range_low', 
        'counter', 'buy', 'sell', 'previous_closing_price', 
        'today_closing_price', 'volume_traded', 'dividend_mk', 'dividend_yield_pct',
        'earnings_yield_pct', 'pe_ratio', 'pbv_ratio', 'market_capitalization_mkmn',
        'profit_after_tax_mkmn', 'num_shares_issue']

In [19]:
# Rename the DataFrame columns to use our predefined column names
# First, let's ensure we're using the right DataFrame and columns
# Our dataframe 'df' already exists, so we'll use it and rename columns to match 'cols'

# Check if columns match in length before renaming
if len(df.columns) == len(cols):
    df.columns = cols
else:
    # Handle the case where column counts don't match
    print(f"Column count mismatch: df has {len(df.columns)} columns, cols has {len(cols)} columns")
    # Assign columns up to the minimum length to avoid errors
    min_length = min(len(df.columns), len(cols))
    df.columns = cols[:min_length] + list(df.columns[min_length:]) if len(df.columns) > min_length else cols
    
# Preview the DataFrame with new column names
print("DataFrame with renamed columns:")
print(df.head())

DataFrame with renamed columns:
   ser_no daily_range_high  daily_range_low  counter    buy   sell  \
0       0           AIRTEL            30.00    30.00  28.81  30.00   
1       1              BHL              NaN      NaN   0.00  12.94   
2       2             FDHB            12.00    12.00   0.00  12.49   
3       3            FMBCH              NaN      NaN  23.00  24.00   
4       4             ICON            12.49    12.49   0.00  12.49   

   previous_closing_price  today_closing_price  volume_traded  dividend_mk  \
0                   30.00                30.00        16444.0         1.25   
1                   12.94                12.94            0.0         0.20   
2                   12.06                12.00      1269828.0         0.00   
3                   24.00                24.00            0.0         0.00   
4                   12.49                12.49        21000.0         0.21   

   dividend_yield_pct  earnings_yield_pct  pe_ratio  pbv_ratio  \
0           

In [20]:
print(df.head())

   ser_no daily_range_high  daily_range_low  counter    buy   sell  \
0       0           AIRTEL            30.00    30.00  28.81  30.00   
1       1              BHL              NaN      NaN   0.00  12.94   
2       2             FDHB            12.00    12.00   0.00  12.49   
3       3            FMBCH              NaN      NaN  23.00  24.00   
4       4             ICON            12.49    12.49   0.00  12.49   

   previous_closing_price  today_closing_price  volume_traded  dividend_mk  \
0                   30.00                30.00        16444.0         1.25   
1                   12.94                12.94            0.0         0.20   
2                   12.06                12.00      1269828.0         0.00   
3                   24.00                24.00            0.0         0.00   
4                   12.49                12.49        21000.0         0.21   

   dividend_yield_pct  earnings_yield_pct  pe_ratio  pbv_ratio  \
0                4.17                4.82   

In [21]:
df.head(16)

,ser_no,daily_range_high,daily_range_low,counter,buy,sell,previous_closing_price,today_closing_price,volume_traded,dividend_mk,dividend_yield_pct,earnings_yield_pct,pe_ratio,pbv_ratio,market_capitalization_mkmn,profit_after_tax_mkmn,num_shares_issue
0,0,AIRTEL,30.00,30.00,28.81,30.00,30.00,30.00,16444.0,1.25,4.17,4.82,20.74,20.95,330000.00,15908.00,1.100000e+10
1,1,BHL,NaN,NaN,0.00,12.94,12.94,12.94,0.0,0.20,1.55,4.29,23.28,1.67,10866.37,466.70,8.397507e+08
2,2,FDHB,12.00,12.00,0.00,12.49,12.06,12.00,1269828.0,0.00,0.00,9.47,10.55,3.65,82812.38,7846.36,6.901031e+09
3,3,FMBCH,NaN,NaN,23.00,24.00,24.00,24.00,0.0,0.00,0.00,-6.24,-16.03,1.02,58998.00,-3679.93,2.458250e+09
4,4,ICON,12.49,12.49,0.00,12.49,12.49,12.49,21000.0,0.21,1.68,8.69,11.51,1.13,83433.20,7248.62,6.680000e+09
5,5,ILLOVO,NaN,NaN,68.50,0.00,80.50,80.50,0.0,0.50,0.62,17.56,5.70,0.81,57432.27,10083.00,7.134444e+08
6,6,MPICO,NaN,NaN,0.00,23.95,24.00,24.00,0.0,0.25,1.04,10.03,9.97,1.41,55153.14,5532.02,2.298047e+09
7,7,NBM,NaN,NaN,0.00,618.75,618.75,618.75,0.0,17.91,2.89,5.92,16.89,2.87,288914.01,17106.00,4.669317e+08
8,8,NBS,21.00,21.00,0.00,21.00,19.44,21.00,32906.0,0.35,1.67,7.29,13.71,3.98,61122.04,4457.61,2.910573e+09
9,9,NICO,48.00,48.00,0.00,48.00,48.00,48.00,5817.0,1.90,3.96,16.00,6.25,1.57,50065.97,8011.00,1.043041e+09


In [22]:
# Extract the first table from the uploaded PDF and save it to CSV

import pdfplumber
import pandas as pd
import re
from pathlib import Path
# from caas_jupyter_tools import display_dataframe_to_user


def clean_cell(x):
    if x is None:
        return None
    # Normalize whitespace and remove stray newlines
    x = re.sub(r'\s+', ' ', str(x)).strip()
    # Replace weird unicode minus or similar artifacts if any
    x = x.replace('–', '-').replace('—', '-')
    return x if x != '' else None

def header_from_rows(rows):
    """
    Pick the first row that looks like a header (most non-empty cells).
    Return (header, start_index_for_data)
    """
    best_idx, best_count = None, -1
    for i, r in enumerate(rows[:5]):  # look at the first few rows
        non_empty = sum(1 for c in r if c is not None and str(c).strip() != '')
        if non_empty > best_count:
            best_count = non_empty
            best_idx = i
    return rows[best_idx], best_idx + 1

first_table_df = None



In [ ]:
# Run the function
df = extract_first_table(pdf_path, out_csv)

In [ ]:
with pdfplumber.open(FILE_PDF) as pdf:
    for page in pdf.pages:
        # Try multiple strategies to improve table detection
        strategies = [
            dict(vertical_strategy="lines", horizontal_strategy="lines"),
            dict(vertical_strategy="lines_strict", horizontal_strategy="lines_strict"),
            dict(vertical_strategy="text", horizontal_strategy="text"),
        ]
        tables = []
        for ts in strategies:
            try:
                t = page.extract_tables(table_settings=ts)
                if t:
                    tables.extend(t)
            except Exception as e:
                # Continue trying with other strategies
                pass
        if tables:
            # Use the first detected table on the first page that has any tables
            raw = tables[0]
            # Clean cells
            rows = [[clean_cell(c) for c in row] for row in raw]
            # Drop completely empty rows
            rows = [row for row in rows if any(cell is not None for cell in row)]
            if not rows:
                continue
            header, start_idx = header_from_rows(rows)
            # If header has duplicates or Nones, generate generic names
            cols = []
            seen = {}
            for i, name in enumerate(header):
                name = name or f"col_{i+1}"
                name = re.sub(r'\s+', ' ', name).strip()
                if name in seen:
                    seen[name] += 1
                    name = f"{name}_{seen[name]}"
                else:
                    seen[name] = 1
                cols.append(name)
            data = rows[start_idx:]
            # Normalize row lengths to header length
            norm_data = []
            for r in data:
                if len(r) < len(cols):
                    r = r + [None] * (len(cols) - len(r))
                elif len(r) > len(cols):
                    r = r[:len(cols)]
                norm_data.append(r)
            df = pd.DataFrame(norm_data, columns=cols)
            # Drop rows that are entirely NaN
            df = df.dropna(how="all")
            # Keep a copy of the first table only
            first_table_df = df
            break

# If we didn't manage to extract a table, create an empty placeholder DataFrame
if first_table_df is None:
    first_table_df = pd.DataFrame()


In [ ]:
headers 

In [ ]:
first_table_df.shape

In [23]:
import pandas as pd
import numpy as np
from pathlib import Path
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

In [24]:
DIR_WORKSPACE = Path.cwd().parent
DIR_DATA = DIR_WORKSPACE / "data"
# DIR_REPORTS_CSV = DIR_DATA / "csv_files"
DIR_OUTPUT = DIR_DATA / "output_combined_data"

In [25]:
DIR_OUTPUT

PosixPath('/home/ira2351696/Desktop/DSCBI/mse-api-assignment/data/output_combined_data')

In [26]:
df = pd.read_csv(DIR_OUTPUT / "combined_csv.csv")
df.head()

,counter_id,daily_range_high,daily_range_low,counter,buy_price,sell_price,previous_closing_price,today_closing_price,volume_traded,dividend_mk,dividend_yield_pct,earnings_yield_pct,pe_ratio,pbv_ratio,market_capitalization_mkmn,profit_after_tax_mkmn,num_shares_issue,trade_date,print_time
0,1.0,138.97,137.94,AIRTEL,0.0,137.96,138.54,137.98,79004.0,2.0,1.45,2.81,35.53,47.26,1517780.0,42722.11,1.100000e+10,2025-09-19,14:37:47
1,2.0,15.02,15.02,BHL,15.02,0.0,15.02,15.02,7042.0,0.0,0.00,-1.55,-64.44,1.36,88291.39,-1370.11,5.878255e+09,2025-09-19,14:37:47
2,3.0,637.95,637.91,FDHB,635.0,637.92,637.95,637.93,101411.0,4.73,0.74,1.68,59.44,45.2,4402374.87,74063.00,6.901031e+09,2025-09-19,14:37:47
3,4.0,1897.99,1897.99,FMBCH,1897.99,0.0,1897.99,1897.99,15133.0,3.64,0.19,2.53,39.45,14.18,4665733.92,118254.74,2.458250e+09,2025-09-19,14:37:47
4,5.0,17.94,17.94,ICON,0.0,17.94,17.94,17.94,18168.0,0.29,1.62,20.38,4.91,0.82,119839.2,24424.49,6.680000e+09,2025-09-19,14:37:47


In [27]:
df.counter.unique()

array(['AIRTEL', 'BHL', 'FDHB', 'FMBCH', 'ICON', 'ILLOVO', 'MPICO', 'NBM',
       'NBS', 'NICO', 'NITL', 'OMU', 'PCL', 'STANDARD', 'SUNBIRD', 'TNM',
       'OML', 'MPICO CD', 'NBM CD', 'NICO CD', 'PCL CD', 'Standard CD',
       'Sunbird CD', 'TNM CD', 'MPICO TS', 'BHL CD', 'BHL XD', 'NITL XD',
       'NITL CD', 'NBS TS', 'NICO TS', 'Sunbird', 'PCL TS', 'Sunbird H',
       'SUNBIRD H', 'StandardCD', 'SUNBIRDCD', 'NITL TS', 'SUNBIRD TS',
       'TNM TS', 'NBM XD', 'StandardTS', 'TNM TS XD', 'TNM TS CD',
       'FMB CA', 'Sunbird XD', 'PCL XD', 'MPICO XD', 'NICO XD',
       'Standard XD', 'TNM XD', 'FMBTS CA XD', 'Sunbird TS',
       'FMBTS CA CD', 'SunbirdXD TS', 'SunbirdCD TS', 'NBS-LA*'],
      dtype=object)

In [28]:
companies = {
    "AIRTEL": "Airtel Malawi plc",
    "BHL": "Blantyre Hotels Plc",
    "FDHB": "FDH Bank plc",
    "FMBCH": "FMB Capital Holdings plc",
    "ICON": "ICON Properties plc",
    "ILLOVO": "Illovo Sugar Malawi plc",
    "MPICO": "Malawi Property Investment Company plc",
    "NBM": "National Bank of Malawi",
    "NBS": "NBS Bank plc",
    "NICO": "NICO Holdings plc",
    "NITL": "National Investment Trust Plc",
    "OMU": "Old Mutual Limited",
    "PCL": "Press Corporation plc",
    "STANDARD": "Standard Bank Malawi plc",
    "SUNBIRD": "Sunbird Tourism plc",
    "TNM": "Telekom Networks Malawi plc",
    "OML": "Old Mutual Limited",
    "MPICO CD": "Malawi Property Investment Company plc",
    "NBM CD": "National Bank of Malawi",
    "NICO CD": "NICO Holdings plc",
    "PCL CD": "Press Corporation plc",
    "Standard CD": "Standard Bank Malawi plc",
    "Sunbird CD": "Sunbird Tourism plc",
    "TNM CD": "Telekom Networks Malawi plc",
    "MPICO TS": "Malawi Property Investment Company plc",
    "BHL CD": "Blantyre Hotels Plc",
    "BHL XD": "Blantyre Hotels Plc",
    "NITL XD": "National Investment Trust Plc",
    "NITL CD": "National Investment Trust Plc",
    "NBS TS": "NBS Bank plc",
    "NICO TS": "NICO Holdings plc",
    "Sunbird": "Sunbird Tourism plc",
    "PCL TS": "Press Corporation plc",
    "Sunbird H": "Sunbird Tourism plc",
    "SUNBIRD H": "Sunbird Tourism plc",
    "StandardCD": "Standard Bank Malawi plc",
    "SUNBIRDCD": "Sunbird Tourism plc",
    "NITL TS": "National Investment Trust Plc",
    "SUNBIRD TS": "Sunbird Tourism plc",
    "TNM TS": "Telekom Networks Malawi plc",
    "NBM XD": "National Bank of Malawi",
    "StandardTS": "Standard Bank Malawi plc",
    "TNM TS XD": "Telekom Networks Malawi plc",
    "TNM TS CD": "Telekom Networks Malawi plc",
    "FMB CA": "FMB Capital Holdings plc",
    "Sunbird XD": "Sunbird Tourism plc",
    "PCL XD": "Press Corporation plc",
    "MPICO XD": "Malawi Property Investment Company plc",
    "NICO XD": "NICO Holdings plc",
    "Standard XD": "Standard Bank Malawi plc",
    "TNM XD": "Telekom Networks Malawi plc",
    "FMBTS CA XD": "FMB Capital Holdings plc",
    "Sunbird TS": "Sunbird Tourism plc",
    "FMBTS CA CD": "FMB Capital Holdings plc",
    "SunbirdXD TS": "Sunbird Tourism plc",
    "SunbirdCD TS": "Sunbird Tourism plc",
    "NBS-LA*": "NBS Bank plc",
}

In [29]:
def company(row):
    return companies[row["counter"]].upper()


df["name"] = df.apply(company, axis=1)
df = df.rename(columns={"counter": "ticker"})
df.head()

,counter_id,daily_range_high,daily_range_low,ticker,buy_price,sell_price,previous_closing_price,today_closing_price,volume_traded,dividend_mk,dividend_yield_pct,earnings_yield_pct,pe_ratio,pbv_ratio,market_capitalization_mkmn,profit_after_tax_mkmn,num_shares_issue,trade_date,print_time,name
0,1.0,138.97,137.94,AIRTEL,0.0,137.96,138.54,137.98,79004.0,2.0,1.45,2.81,35.53,47.26,1517780.0,42722.11,1.100000e+10,2025-09-19,14:37:47,AIRTEL MALAWI PLC
1,2.0,15.02,15.02,BHL,15.02,0.0,15.02,15.02,7042.0,0.0,0.00,-1.55,-64.44,1.36,88291.39,-1370.11,5.878255e+09,2025-09-19,14:37:47,BLANTYRE HOTELS PLC
2,3.0,637.95,637.91,FDHB,635.0,637.92,637.95,637.93,101411.0,4.73,0.74,1.68,59.44,45.2,4402374.87,74063.00,6.901031e+09,2025-09-19,14:37:47,FDH BANK PLC
3,4.0,1897.99,1897.99,FMBCH,1897.99,0.0,1897.99,1897.99,15133.0,3.64,0.19,2.53,39.45,14.18,4665733.92,118254.74,2.458250e+09,2025-09-19,14:37:47,FMB CAPITAL HOLDINGS PLC
4,5.0,17.94,17.94,ICON,0.0,17.94,17.94,17.94,18168.0,0.29,1.62,20.38,4.91,0.82,119839.2,24424.49,6.680000e+09,2025-09-19,14:37:47,ICON PROPERTIES PLC


In [30]:
df_prices = df.copy()
df_prices.head()

,counter_id,daily_range_high,daily_range_low,ticker,buy_price,sell_price,previous_closing_price,today_closing_price,volume_traded,dividend_mk,dividend_yield_pct,earnings_yield_pct,pe_ratio,pbv_ratio,market_capitalization_mkmn,profit_after_tax_mkmn,num_shares_issue,trade_date,print_time,name
0,1.0,138.97,137.94,AIRTEL,0.0,137.96,138.54,137.98,79004.0,2.0,1.45,2.81,35.53,47.26,1517780.0,42722.11,1.100000e+10,2025-09-19,14:37:47,AIRTEL MALAWI PLC
1,2.0,15.02,15.02,BHL,15.02,0.0,15.02,15.02,7042.0,0.0,0.00,-1.55,-64.44,1.36,88291.39,-1370.11,5.878255e+09,2025-09-19,14:37:47,BLANTYRE HOTELS PLC
2,3.0,637.95,637.91,FDHB,635.0,637.92,637.95,637.93,101411.0,4.73,0.74,1.68,59.44,45.2,4402374.87,74063.00,6.901031e+09,2025-09-19,14:37:47,FDH BANK PLC
3,4.0,1897.99,1897.99,FMBCH,1897.99,0.0,1897.99,1897.99,15133.0,3.64,0.19,2.53,39.45,14.18,4665733.92,118254.74,2.458250e+09,2025-09-19,14:37:47,FMB CAPITAL HOLDINGS PLC
4,5.0,17.94,17.94,ICON,0.0,17.94,17.94,17.94,18168.0,0.29,1.62,20.38,4.91,0.82,119839.2,24424.49,6.680000e+09,2025-09-19,14:37:47,ICON PROPERTIES PLC


In [31]:
df_prices = df.copy()
df_prices.to_csv(DIR_OUTPUT / "master_dataset.csv")

In [32]:
df = pd.read_csv(DIR_OUTPUT/"site_data.csv")
print(df.head())
ids = {}
for ticker, id in zip(df["Company Name"].values, df["ISIN"].values):
    ids[ticker] = id
df_prices["counter_id"] = df_prices["name"].apply(lambda x: ids[x])
# df_prices.head()

prices = {}
for ticker, price in zip(df["Company Name"].values, df["Listing Price"].values):
    prices[ticker] = price
df_prices["open_mwk"] = df_prices["name"].apply(lambda x: prices[x])
df_prices.head()

               Company Name  Symbol          ISIN  Listing Price Date Listed
0         AIRTEL MALAWI PLC  AIRTEL  MWAIRT001156          12.69   24-Feb-20
1       BLANTYRE HOTELS PLC     BHL  MWBHL0010029           0.84   25-Mar-97
2              FDH BANK PLC    FDHB  MWFDHB001166          10.00   03-Aug-20
3  FMB CAPITAL HOLDINGS PLC   FMBCH  MWFMB0010138          45.01   18-Sep-17
4       ICON PROPERTIES PLC    ICON  MWICON001146           8.75   21-Jan-19


,counter_id,daily_range_high,daily_range_low,ticker,buy_price,sell_price,previous_closing_price,today_closing_price,volume_traded,dividend_mk,...,earnings_yield_pct,pe_ratio,pbv_ratio,market_capitalization_mkmn,profit_after_tax_mkmn,num_shares_issue,trade_date,print_time,name,open_mwk
0,MWAIRT001156,138.97,137.94,AIRTEL,0.0,137.96,138.54,137.98,79004.0,2.0,...,2.81,35.53,47.26,1517780.0,42722.11,1.100000e+10,2025-09-19,14:37:47,AIRTEL MALAWI PLC,12.69
1,MWBHL0010029,15.02,15.02,BHL,15.02,0.0,15.02,15.02,7042.0,0.0,...,-1.55,-64.44,1.36,88291.39,-1370.11,5.878255e+09,2025-09-19,14:37:47,BLANTYRE HOTELS PLC,0.84
2,MWFDHB001166,637.95,637.91,FDHB,635.0,637.92,637.95,637.93,101411.0,4.73,...,1.68,59.44,45.2,4402374.87,74063.00,6.901031e+09,2025-09-19,14:37:47,FDH BANK PLC,10.00
3,MWFMB0010138,1897.99,1897.99,FMBCH,1897.99,0.0,1897.99,1897.99,15133.0,3.64,...,2.53,39.45,14.18,4665733.92,118254.74,2.458250e+09,2025-09-19,14:37:47,FMB CAPITAL HOLDINGS PLC,45.01
4,MWICON001146,17.94,17.94,ICON,0.0,17.94,17.94,17.94,18168.0,0.29,...,20.38,4.91,0.82,119839.2,24424.49,6.680000e+09,2025-09-19,14:37:47,ICON PROPERTIES PLC,8.75


In [33]:
# extract only the required fields
df_prices = df_prices[
    [
        "counter_id",
        "trade_date",
        "open_mwk",
        "daily_range_high",
        "daily_range_low",
        "today_closing_price",
        "volume_traded",
    ]
]
df_prices.columns = [
    "counter_id",
    "trade_date",
    "open_mwk",
    "high_mwk",
    "low_mwk",
    "close_mwk",
    "volume",
]

In [34]:
# df_prices.to_csv(DIR_OUTPUT / "master_dataset.csv")
df_prices["volume"] = df_prices["volume"].astype(float)
df_prices["trade_date"] = pd.to_datetime(df_prices["trade_date"])
df_prices["close_mwk"] = df_prices["close_mwk"].astype(float)
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26810 entries, 0 to 26809
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   counter_id  26810 non-null  object        
 1   trade_date  26810 non-null  datetime64[ns]
 2   open_mwk    26810 non-null  float64       
 3   high_mwk    11207 non-null  float64       
 4   low_mwk     11207 non-null  float64       
 5   close_mwk   25151 non-null  float64       
 6   volume      25147 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.4+ MB


In [36]:
load_dotenv()

PGHOST = os.getenv("PGHOST")
PGPORT = os.getenv("PGPORT", "5432")
PGDATABASE = os.getenv("PGDATABASE")
PGUSER = os.getenv("PGUSER")
PGPASSWORD = os.getenv("PGPASSWORD")

In [37]:
# Create SQLAlchemy engine (no password needed for local connections)
connection_string = (
    f"postgresql+psycopg2://{PGUSER}:{PGPASSWORD}@{PGHOST}:{PGPORT}/{PGDATABASE}"
)

In [20]:
engine = create_engine(
    connection_string,
    pool_pre_ping=True,
)
# create a table
query = """create table if not exists counters (counter_id text primary key, ticker text not null, name text not null, listing_price real not null, date_listed date not null);"""
with engine.begin() as conn:
    conn.execute(text(query))

In [21]:
df = pd.read_csv(DIR_OUTPUT/"site_data.csv")
df.head()
df.columns = ["name", "ticker", "counter_id", "listing_price", "date_listed"]
df = df[["counter_id", "ticker", "name", "listing_price", "date_listed"]]
df["date_listed"] = pd.to_datetime(df["date_listed"]).dt.date
df["listing_price"] = df["listing_price"].astype(float)
df.head()

/tmp/ipykernel_64291/16428772.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date_listed"] = pd.to_datetime(df["date_listed"]).dt.date


,counter_id,ticker,name,listing_price,date_listed
0,MWAIRT001156,AIRTEL,AIRTEL MALAWI PLC,12.69,2020-02-24
1,MWBHL0010029,BHL,BLANTYRE HOTELS PLC,0.84,1997-03-25
2,MWFDHB001166,FDHB,FDH BANK PLC,10.00,2020-08-03
3,MWFMB0010138,FMBCH,FMB CAPITAL HOLDINGS PLC,45.01,2017-09-18
4,MWICON001146,ICON,ICON PROPERTIES PLC,8.75,2019-01-21


In [ ]:
df.to_sql("counters", engine, if_exists="append", index=False)

In [20]:
# Create prices_daily table
query = """create table if not exists prices (
    counter_id text REFERENCES counters(counter_id),
    trade_date date,
    open_mwk real,
    high_mwk real,
    low_mwk real,
    close_mwk real,
    volume real
    );"""
with engine.begin() as conn:
    conn.execute(text(query))

In [21]:
df_prices.to_sql("prices", engine, if_exists="append", index=False)

810